## Linking a dataset of real historical persons

In this example, we deduplicate a more realistic dataset. The data is based on historical persons scraped from wikidata. Duplicate records are introduced with a variety of errors introduced.


<a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/splink4_dev/docs/demos/examples/duckdb/deduplicate_50k_synthetic.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install git+https://github.com/moj-analytical-services/splink.git@splink4_dev

In [2]:
from splink import splink_datasets

df = splink_datasets.historical_50k
df.head()

downloading: https://raw.githubusercontent.com/moj-analytical-services/splink_datasets/master/data/historical_figures_with_errors_50k.parquet


  download progress: 100 %	(==========)


,unique_id,cluster,full_name,first_and_surname,first_name,surname,dob,birth_place,postcode_fake,gender,occupation
0,Q2296770-1,Q2296770,"thomas clifford, 1st baron clifford of chudleigh",thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,male,politician
1,Q2296770-2,Q2296770,thomas of chudleigh,thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,male,politician
2,Q2296770-3,Q2296770,tom 1st baron clifford of chudleigh,tom chudleigh,tom,chudleigh,1630-08-01,devon,tq13 8df,male,politician
3,Q2296770-4,Q2296770,thomas 1st chudleigh,thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8hu,None,politician
4,Q2296770-5,Q2296770,"thomas clifford, 1st baron chudleigh",thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,None,politician


In [3]:
from splink import DuckDBAPI
from splink.exploratory import profile_columns

db_api = DuckDBAPI()
profile_columns(df, db_api, column_expressions=["first_name", "substr(surname,1,2)"])

alt.VConcatChart(...)

In [4]:
from splink import DuckDBAPI, block_on
from splink.blocking_analysis import (
    cumulative_comparisons_to_be_scored_from_blocking_rules_chart,
)

blocking_rules =  [block_on("first_name", "surname"),
        block_on("surname", "dob"),
        block_on("first_name", "dob"),
        block_on("postcode_fake", "first_name")]

db_api = DuckDBAPI()

cumulative_comparisons_to_be_scored_from_blocking_rules_chart(
    table_or_tables=df,
    blocking_rules=blocking_rules,
    db_api=db_api,
    link_type="dedupe_only",
)

alt.Chart(...)

In [5]:
import splink.comparison_library as cl
import splink.comparison_template_library as ctl
from splink import Linker, SettingsCreator

settings = SettingsCreator(
    link_type="dedupe_only",
    blocking_rules_to_generate_predictions=blocking_rules,
    comparisons=[
        ctl.NameComparison("first_name").configure(term_frequency_adjustments=True),
        ctl.NameComparison("surname").configure(term_frequency_adjustments=True),
        ctl.DateComparison(
            "dob",
            input_is_string=True,
            datetime_metrics=["month", "year", "year"],
            datetime_thresholds=[1, 1, 10],
        ),
        # TODO: Restore ctl.PostcodeComparison level here
        cl.LevenshteinAtThresholds("postcode_fake"),
        cl.ExactMatch("birth_place").configure(term_frequency_adjustments=True),
        cl.ExactMatch("occupation").configure(term_frequency_adjustments=True),
    ],
    retain_intermediate_calculation_columns=True,
)

linker = Linker(df, settings, database_api=db_api)

In [6]:
linker.training.estimate_probability_two_random_records_match(
    [
        "l.first_name = r.first_name and l.surname = r.surname and l.dob = r.dob",
        "substr(l.first_name,1,2) = substr(r.first_name,1,2) and l.surname = r.surname and substr(l.postcode_fake,1,2) = substr(r.postcode_fake,1,2)",
        "l.dob = r.dob and l.postcode_fake = r.postcode_fake",
    ],
    recall=0.6,
)

Probability two random records match is estimated to be  0.000136.
This means that amongst all possible pairwise record comparisons, one in 7,362.31 are expected to match.  With 1,279,041,753 total possible comparisons, we expect a total of around 173,728.33 matching pairs


In [7]:
linker.training.estimate_u_using_random_sampling(max_pairs=5e6)

----- Estimating u probabilities using random sampling -----



Estimated u probabilities using random sampling



Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - postcode_fake (no m values are trained).
    - birth_place (no m values are trained).
    - occupation (no m values are trained).


In [8]:
training_blocking_rule = block_on("first_name", "surname")
training_session_names = linker.training.estimate_parameters_using_expectation_maximisation(
    training_blocking_rule
)


----- Starting EM training session -----



Estimating the m probabilities of the model by blocking on:
(l."first_name" = r."first_name") AND (l."surname" = r."surname")

Parameter estimates will be made for the following comparison(s):
    - dob
    - postcode_fake
    - birth_place
    - occupation

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name
    - surname


Iteration 1: Largest change in params was -0.524 in probability_two_random_records_match


Iteration 2: Largest change in params was -0.0289 in probability_two_random_records_match


Iteration 3: Largest change in params was -0.0106 in the m_probability of birth_place, level `All other comparisons`


...

Iteration 8: Largest change in params was -0.000267 in the m_probability of dob, level `Abs difference of 'transformed dob <= 10 year'`


Iteration 9: Largest change in params was -0.000144 in the m_probability of dob, level `Abs difference of 'transformed dob <= 10 year'`


Iteration 10: Largest change in params was -7.75e-05 in the m_probability of dob, level `Abs difference of 'transformed dob <= 10 year'`



EM converged after 10 iterations



Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).


In [9]:
training_blocking_rule = block_on("dob")
training_session_dob = linker.training.estimate_parameters_using_expectation_maximisation(
    training_blocking_rule
)


----- Starting EM training session -----



Estimating the m probabilities of the model by blocking on:
l."dob" = r."dob"

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - postcode_fake
    - birth_place
    - occupation

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob


Iteration 1: Largest change in params was -0.362 in the m_probability of first_name, level `Exact match on first_name`


Iteration 2: Largest change in params was 0.0336 in the m_probability of first_name, level `All other comparisons`


Iteration 3: Largest change in params was 0.00479 in the m_probability of first_name, level `All other comparisons`


Iteration 4: Largest change in params was 0.00106 in the m_probability of surname, level `All other comparisons`


Iteration 5: Largest change in params was 0.000254 in the m_probability of surname, level `All other comparisons`


Iteration 6: Largest change in params was 5.93e-05 in the m_probability of surname, level `All other comparisons`



EM converged after 6 iterations



Your model is fully trained. All comparisons have at least one estimate for their m and u values


The final match weights can be viewed in the match weights chart:


In [10]:
linker.visualisations.match_weights_chart()

alt.VConcatChart(...)

In [11]:
linker.evaluation.unlinkables_chart()

alt.LayerChart(...)

In [12]:
df_predict = linker.inference.predict()
df_e = df_predict.as_pandas_dataframe(limit=5)
df_e

,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,tf_first_name_l,tf_first_name_r,bf_first_name,...,bf_birth_place,bf_tf_adj_birth_place,occupation_l,occupation_r,gamma_occupation,tf_occupation_l,tf_occupation_r,bf_occupation,bf_tf_adj_occupation,match_key
0,-15.829333,0.000017,Q7528564-9,Q75867928-1,sir,sir,3,0.024985,0.024985,38.34881,...,0.157016,1.0,historian,military officer,0,0.012456,0.010756,0.105028,1.0,0
1,-15.829333,0.000017,Q7528564-9,Q75867928-2,sir,sir,3,0.024985,0.024985,38.34881,...,0.157016,1.0,historian,military officer,0,0.012456,0.010756,0.105028,1.0,0
2,-15.829333,0.000017,Q7528564-9,Q75867928-3,sir,sir,3,0.024985,0.024985,38.34881,...,0.157016,1.0,historian,military officer,0,0.012456,0.010756,0.105028,1.0,0
3,-15.829333,0.000017,Q7528564-9,Q75867928-4,sir,sir,3,0.024985,0.024985,38.34881,...,0.157016,1.0,historian,military officer,0,0.012456,0.010756,0.105028,1.0,0
4,-15.829333,0.000017,Q7528564-9,Q75867928-6,sir,sir,3,0.024985,0.024985,38.34881,...,0.157016,1.0,historian,military officer,0,0.012456,0.010756,0.105028,1.0,0


You can also view rows in this dataset as a waterfall chart as follows:


In [13]:

records_to_plot = df_e.to_dict(orient="records")
linker.visualisations.waterfall_chart(records_to_plot, filter_nulls=False)

alt.LayerChart(...)

In [14]:
clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(
    df_predict, threshold_match_probability=0.95
)

Completed iteration 1, root rows count 623


Completed iteration 2, root rows count 100


Completed iteration 3, root rows count 22


Completed iteration 4, root rows count 4


Completed iteration 5, root rows count 0


In [15]:
from IPython.display import IFrame

linker.visualisations.cluster_studio_dashboard(
    df_predict,
    clusters,
    "dashboards/50k_cluster.html",
    sampling_method="by_cluster_size",
    overwrite=True,
)


IFrame(src="./dashboards/50k_cluster.html", width="100%", height=1200)

In [16]:
linker.evaluation.accuracy_analysis_from_labels_column(
    "cluster", output_type="roc", match_weight_round_to_nearest=0.02
)

alt.Chart(...)

In [17]:
records = linker.evaluation.prediction_errors_from_labels_column(
    "cluster",
    threshold=0.999,
    include_false_negatives=False,
    include_false_positives=True,
).as_record_dict()
linker.visualisations.waterfall_chart(records)

alt.LayerChart(...)

In [18]:
# Some of the false negatives will be because they weren't detected by the blocking rules
records = linker.evaluation.prediction_errors_from_labels_column(
    "cluster",
    threshold=0.5,
    include_false_negatives=True,
    include_false_positives=False,
).as_record_dict(limit=50)

linker.visualisations.waterfall_chart(records)

alt.LayerChart(...)